In [ ]:
import time
import pandas as pd
import csv
from instaloader import Instaloader, Profile, exceptions
from datetime import datetime, timedelta
from itertools import dropwhile, takewhile
from os import listdir, mkdir
import openpyxl

In [ ]:
def descarga_posts_entre_fechas(str_profile, dt_fecha_inicio, dt_fecha_final, str_directorio):
    """
    Guarda los posts entre las fechas indicadas (ambas 
    inclusive) y genera un archivo resumen en formato csv
    """

    # Generar instancia de Instaloader
    L = Instaloader(compress_json=False, save_metadata=False) # No descargar la informacion sobre el nodo de Instagram
    profile = Profile.from_username(L.context, str_profile) # Descargar el perfil
    posts = profile.get_posts() # Recupera todos los posts del perfil cargado

    # Campos del csv
    campos_csv = ['num', 'post', 'shortcode', 'fecha', 'hora', 'video', 'likes', 'comentarios', 'texto', 'hashtags']
    lista_filas = []

    # Contador de posts
    n = 0

    # Descarga iterativamente los posts del perfil dentro del intervalo de fechas indicado
    for post in takewhile(lambda p: p.date > dt_fecha_inicio, dropwhile(lambda p: p.date > dt_fecha_final + timedelta(1), posts)):
        try:
            L.download_post(post, str_directorio) # Intenta descargar el post en el directorio
        except instaloader.exceptions.InstaloaderException:
            print(f"Error al descargar la imagen: {post}") # A veces no puede acceder al post correctamente

        n += 1 # Contador para saber cuantos posts ha publicado el usuario en el intervalo de fechas

        # Generar el archivo resumen
        fila = {}
        fila['num'] = 0 # Inicializar el indice de la fila
        fila['post'] = post.date.strftime('%Y-%m-%d_%H-%M-%S_UTC')
        fila['shortcode'] = post.shortcode
        dt_post_corregido = post.date + timedelta(hours=1) # Ajustamos la hora de UTC a horario de España
        fila['fecha'] = dt_post_corregido.strftime('%Y-%m-%d')
        fila['hora'] = dt_post_corregido.strftime('%H:%M:%S')
        if post.is_video:
            fila['video'] = 1
        else:
            fila['video'] = 0
        fila['likes'] = post.likes
        fila['comentarios'] = post.comments
        if post.caption is not None:
            fila['texto'] = post.caption.replace('\n', ' ').replace('\r', ' ')
            fila['hashtags'] = '; '.join(list(set(post.caption_hashtags)))
        
        lista_filas.append(fila)

        # Esto es para no saturar el servidor y evitar la desconexion
        if n % 100 == 0:
            print(f'{'-'*10}DURMIENDO 30 SEGUNDOS{'-'*10}')
            time.sleep(30)

    # Actualizar el indice de los posts
    for n_fila, fila in enumerate(reversed(lista_filas)):
        fila['num'] = n_fila + 1

    # Guardamos el resumen de los posts descargados del usuario entre las fechas indicadas en un archivo csv
    with open(str_directorio + '/0_' + str_profile + '_resumen.csv', 'w', encoding='UTF-16') as archivo:
        writer = csv.DictWriter(archivo, fieldnames=campos_csv, delimiter='\t', lineterminator='\n')
        writer.writeheader()
        writer.writerows(reversed(lista_filas))
        
    return n
        
def seguidores(str_profile):
    """Devuelve el numero de seguidores de un perfil de Instagram"""
    L = Instaloader()
    profile = Profile.from_username(L.context, str_profile)
    return profile.followers

In [ ]:
# Archivo con los usuarios a descargar
archivo_excel = 'Ganadores Grammys 19 - 23.xlsx'
dataframe = pd.read_excel(archivo_excel)

users = []

# Creamos la lista con los usuarios de Instagram pendientes de descargar 
# (Esto se ejecuta despues de que haya dado error por desconexion del servidor)
for indice_fila, fila in dataframe.iterrows():
    user = fila['INSTAGRAM']
    descargado = fila['DESCARGADO']

    if descargado == 0:
        users.append(user)

len(users)

Descarga de posts

In [ ]:
for user in users:
    str_profile = str(user)

    # Periodo a descargar (en este caso el año entre las dos ultimas ediciones de los Premios Grammy)
    inicio = datetime(2023, 2, 6)
    final = datetime(2024, 2, 4)
    if str_profile not in listdir('.'):
        mkdir(str_profile)

    print('Descargando posts en el directorio {}...'.format(str_profile))
    n_posts = descarga_posts_entre_fechas(str_profile, inicio, final, str_profile)
    followers = seguidores(str_profile)
    print('OPERACIÓN FINALIZADA\n - Se han descargado {} posts de la cuenta de Instagram de {}'.format(n_posts, str_profile))
    
    # Abrimos el archivo Excel con los usuarios de Instagram,
    # modificamos la columna "DESCARGADO" y añadimos el numero de posts y seguidores
    wb = openpyxl.load_workbook('Ganadores Grammys 19 - 23.xlsx')
    hoja = wb.active

    for index, row in enumerate(hoja.iter_rows(min_row = 1, values_only = True)):
        if row[1] == str_profile:
            hoja.cell(row=index+1, column=5, value=1)
            print(f' - La cuenta de Instagram {str_profile} ha sido marcada como descargada')
            hoja.cell(row=index+1, column=6, value=n_posts)
            hoja.cell(row=index+1, column=7, value=followers)
            print(f'- La cuenta de Instagram {str_profile} tiene {followers} seguidores\n')

wb.save('Ganadores Grammys 19 - 23.xlsx')